## Network_graph

- 흩어져있는 1:1관계 데이터를 1:N 형태(리스트)로 묶어서 연결 구조와 중심을 파악하는 그래프 과정

| 함수                        | 설명                                          |
| :------------------------ | :------------------------------------------ |
| **`collect_set(col)`**    | 그룹별로 데이터를 긁어모아 **중복 없는 배열(Array)** 로 만듦.     |
| **`concat_ws(sep, col)`** | 배열을 특정 구분자(`,`)로 이어 붙여 **문자열(String)** 로 만듦. |
| **`size(col)`**           | 배열이나 맵의 **크기(개수)** 를 셈.                      |


In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [3]:
spark=(
    SparkSession
    .builder
    .appName("network-graph")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/30 09:43:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df=spark.read.csv(
    "file:///workspace/data/order_products.csv",
    header=True,
    inferSchema=True
)
df.show()
df.printSchema()

+--------+-------+
|order_id|product|
+--------+-------+
|    1001|  Apple|
|    1001| Banana|
|    1001|   Milk|
|    1001|  Bread|
|    1002|  Apple|
|    1002|   Milk|
|    1003| Banana|
|    1003|   Milk|
|    1003|   Eggs|
|    1004|  Apple|
|    1004|  Bread|
|    1004| Butter|
|    1004|   Milk|
|    1004|   Eggs|
|    1005|   Beer|
|    1005|  Chips|
|    1006|  Apple|
|    1006| Banana|
|    1006|   Milk|
|    1006|  Bread|
+--------+-------+
only showing top 20 rows

root
 |-- order_id: integer (nullable = true)
 |-- product: string (nullable = true)



In [8]:
# step1 : 대상별 관계 리스트 만들기 
# 분석하고 싶은 주체(key)를 기준으로 , 연관된 대상을 모으기 
# 결과는 주체 |[대상1,대상2,대상3..] 배열 타입
# 이 주체가 누구와 연결되어있는가를 한줄로 요약가능 
# collect_set

In [9]:
# 주문별 장바구니 만들기 

In [17]:
basket_df=(
    df
    .groupBy("order_id")
    .agg(F.collect_set("product").alias("products"))
)
basket_df.show(truncate=False)

+--------+------------------------------------------+
|order_id|products                                  |
+--------+------------------------------------------+
|1005    |[Chips, Beer]                             |
|1002    |[Milk, Apple]                             |
|1001    |[Milk, Bread, Apple, Banana]              |
|1006    |[Milk, Eggs, Bread, Butter, Apple, Banana]|
|1003    |[Milk, Eggs, Banana]                      |
|1004    |[Milk, Eggs, Bread, Butter, Apple]        |
+--------+------------------------------------------+



In [19]:
# 만약 csv로 저장하려면 
# 리스트 타입은 csv로 저장이 안되기때문에, 문자열로 바꿔야한다!
# concat_ws(sep, col)

In [25]:
basket_df=basket_df.withColumn(
    "products",
    F.concat_ws(",",F.col("products"))
)
# 파티션 1개로 합쳐서 csv저장
basket_df.coalesce(1).write.option("header",True).csv("data/order_output")

In [21]:
# 다시 파일 가져와서 size측정

In [24]:
df=spark.read.csv(
    "file:///workspace/data/order_output/order_product_output.csv",
    header=True,
    inferSchema=True
)
df.show()
df.printSchema()

+--------+--------------------+
|order_id|            products|
+--------+--------------------+
|    1005|          Chips,Beer|
|    1002|          Milk,Apple|
|    1001|Milk,Bread,Apple,...|
|    1006|Milk,Eggs,Bread,B...|
|    1003|    Milk,Eggs,Banana|
|    1004|Milk,Eggs,Bread,B...|
+--------+--------------------+

root
 |-- order_id: integer (nullable = true)
 |-- products: string (nullable = true)



In [30]:
# size를 사용하려면 다시 리스트로 복원 
df=df.withColumn(
    "products_count",
    F.size(F.split(F.col("products"),","))
)
df.orderBy(F.desc("products_count")).show()

+--------+--------------------+--------------+
|order_id|            products|products_count|
+--------+--------------------+--------------+
|    1006|Milk,Eggs,Bread,B...|             6|
|    1004|Milk,Eggs,Bread,B...|             5|
|    1001|Milk,Bread,Apple,...|             4|
|    1003|    Milk,Eggs,Banana|             3|
|    1005|          Chips,Beer|             2|
|    1002|          Milk,Apple|             2|
+--------+--------------------+--------------+



In [31]:
# 만약 저장안해도 된다면 바로 하는방법 !을 하자!!!

In [24]:
basket_df=(
    df
    .groupBy("order_id")
    .agg(
        F.collect_set("product").alias("products"),
        F.size("products").alias("product_count")
    )
    .orderBy(F.col("product_count").desc())
)
basket_df.show(truncate=False)

+--------+------------------------------------------+-------------+
|order_id|products                                  |product_count|
+--------+------------------------------------------+-------------+
|1006    |[Milk, Eggs, Bread, Butter, Apple, Banana]|6            |
|1004    |[Milk, Eggs, Bread, Butter, Apple]        |5            |
|1001    |[Milk, Bread, Apple, Banana]              |4            |
|1003    |[Milk, Eggs, Banana]                      |3            |
|1005    |[Chips, Beer]                             |2            |
|1002    |[Milk, Apple]                             |2            |
+--------+------------------------------------------+-------------+

